<a href="https://colab.research.google.com/github/Raivotar/Kaggle_Competitions/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
train_data = pd.read_csv("https://raw.githubusercontent.com/Raivotar/Kaggle_Competitions/main/Titanic/Dataset/train.csv")
test_data = pd.read_csv("https://raw.githubusercontent.com/Raivotar/Kaggle_Competitions/main/Titanic/Dataset/test.csv")
test_ids = test_data["PassengerId"]

In [3]:
train_data.shape

(891, 12)

In [4]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
train_data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
def clean(data):
  data = data.drop(["Ticket", "Cabin", "Name", "PassengerId"], axis=1)
  cols = ["SibSp", "Parch", "Fare", "Age"]
  for col in cols:
    data[col].fillna(data[col].median(), inplace=True)
  data["Embarked"].fillna("U", inplace=True)
  return data
train_data = clean(train_data)
test_data = clean(test_data)

In [7]:
train_data.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [8]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

cols = ["Sex", "Embarked"]

for col in cols:
  train_data[col] = le.fit_transform(train_data[col])
  test_data[col] = le.transform(test_data[col])
  print(le.classes_)

['female' 'male']
['C' 'Q' 'S' 'U']


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

y = train_data["Survived"]
X = train_data.drop("Survived", axis=1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, y_train)

In [11]:
predictions = clf.predict(X_val)
from sklearn.metrics import accuracy_score
accuracy_score(y_val, predictions)

0.8100558659217877

In [12]:
submission_preds = clf.predict(test_data)

In [13]:
df = pd.DataFrame({"PassengerId": test_ids, "Survived": submission_preds})
df.to_csv("submission.csv", index=False)